<a href="https://colab.research.google.com/github/BognarAndras/corvinus_ds_ws/blob/main/data_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Predictive Analysis- Ingatlan.com adat

### Single case of webscrape

In [1]:
import requests

cookies = {
    'optimizelySegments': '%7B%22653870010%22%3A%22gc%22%2C%22654380236%22%3A%22direct%22%2C%22656860019%22%3A%22false%22%7D',
    'optimizelyEndUserId': 'oeu1439225375810r0.12509430727023318',
    'optimizelyBuckets': '%7B%7D',
    'bbid': '2539656256515775',
    'wb-p-SERVER': 'wwwb-app213',
    'donation-identifier': 'b6e48a16f7b59f1ca2a401bc0914c351',
    'wb-cdx-ui-SERVER': 'wwwb-app205',
}

headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8',
    'accept-language': 'en-US,en;q=0.5',
    'cache-control': 'max-age=0',
    'priority': 'u=0, i',
    'sec-ch-ua': '"Chromium";v="136", "Brave";v="136", "Not.A/Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'none',
    'sec-fetch-user': '?1',
    'sec-gpc': '1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36',
    # 'cookie': 'optimizelySegments=%7B%22653870010%22%3A%22gc%22%2C%22654380236%22%3A%22direct%22%2C%22656860019%22%3A%22false%22%7D; optimizelyEndUserId=oeu1439225375810r0.12509430727023318; optimizelyBuckets=%7B%7D; bbid=2539656256515775; wb-p-SERVER=wwwb-app213; donation-identifier=b6e48a16f7b59f1ca2a401bc0914c351; wb-cdx-ui-SERVER=wwwb-app205',
}

response = requests.get(
    'https://web.archive.org/web/20150810164931/http://ingatlan.com/budapest/kiado+lakas',
    cookies=cookies,
    headers=headers,
)

In [2]:
'property-name' in response.text

True

In [10]:
from bs4 import BeautifulSoup
import pandas as pd
import re

html = response.text

soup = BeautifulSoup(html, "html.parser")

rows = []

for tr in soup.find_all("tr", class_="list-row"):
    try:
        centered_blocks = tr.find_all("td", class_="centered")

        # --- Price ---
        price_block = centered_blocks[0]
        price_text = price_block.get_text(" ", strip=True)

        # Keep only part before 'Ft/hó' (HUF)
        if "Ft" in price_text:
            huf_part = price_text.split("Ft")[0]
        else:
            huf_part = price_text  # fallback

        # Extract digits from that section only
        digits = re.findall(r'\d+', huf_part)
        price = int("".join(digits)) if digits else None

        # --- Size ---
        size_span = centered_blocks[1].find("span", class_="numbers-highlight")
        size = int(re.findall(r'\d+', size_span.text)[0]) if size_span else None

        # --- Rooms ---
        rooms_span = centered_blocks[2].find("span", class_="numbers-highlight")
        rooms = int(re.findall(r'\d+', rooms_span.text)[0]) if rooms_span else None

        # --- Property Name (robust) ---
        address_td = tr.find("td", class_="address")
        if address_td:
            # get all text inside the <a> but ignore <p> content
            a_tag = address_td.find("a")
            if a_tag:
                # remove the <p> tag from the a_tag
                p_tag = a_tag.find("p", class_="property-name")
                if p_tag:
                    p_tag.extract()
                # get remaining text and clean it
                property_name = " ".join(a_tag.stripped_strings)
            else:
                property_name = ""
        else:
            property_name = ""


        # --- Feature detection ---
        from_owner = int("Tulajdonostól" in property_name)
        great_view = int("Panorámás" in property_name)

        # Clean them out of property_name text
        property_name = (property_name
                         .replace("Tulajdonostól!", "")
                         .replace("Panorámás", "")
                         .strip())

        # --- Split into town and address ---
        parts = [p.strip() for p in property_name.split(",", 1)]
        if len(parts) == 2:
            town, address = parts
        else:
            town = parts[0] if parts else ""
            address = ""

        rows.append({
            "price": price,
            "size": size,
            "rooms": rooms,
            "from_owner": from_owner,
            "great_view": great_view,
            "town": town,
            "address": address
        })

    except Exception as e:
        print(f"Skipping row due to error: {e}")

df = pd.DataFrame(rows)
print(df)

     price  size  rooms  from_owner  great_view      town  \
0   115000    37      1           0           0  Budapest   
1   124180    35      1           0           0  Budapest   
2   145000    50      2           1           0  Budapest   
3   130000    40      1           1           0  Budapest   
4   150000    78      3           1           0  Budapest   
5   180000    40      2           1           0  Budapest   
6   803686   155      4           0           1  Budapest   
7    75000    35      1           0           0  Budapest   
8   110000    59      2           0           0  Budapest   
9   100000    40      1           0           0  Budapest   
10  260000    79      3           0           0  Budapest   
11  169900    62      2           0           0  Budapest   
12  160000    80      2           0           0  Budapest   
13  185000    70      1           0           0  Budapest   
14  230000    78      2           0           0  Budapest   
15  130000    50      2 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 106.9 MB/s eta 0:00:00




### Getting full data and export it


34.91.84.138
